In [17]:
import os, sys, csv, re, importlib
from itertools import product
sys.path.insert(0, "..")
from helpers.parsers import split_path

In [4]:
def split_path(path):
    # This regex captures filename after the last backslash
    filename = re.search("(?!\/)(?:.(?!\/))*(?=\.\w*$)", path).group(0)
    path_prefix = re.search("(.*\/)(?!.*\/)", path).group(0)
    
    return path_prefix, filename

In [37]:
def create_event(config, PARAMETERS, CV_SETTINGS):
    # Search for model with Cartisan Product of hyperparameters
    parameter_lists = []
    for parameter in PARAMETERS:
        parameter_lists.append(getattr(config.Hyperparameter, parameter))
    search_space = product(*parameter_lists)
    
    payload_list = []

    for item in search_space:
        payload = {}
        for key, value in zip(PARAMETERS, list(item)):
            payload[key.lower()] = value
        payload_list.append(payload)

    with open("parameter_mappings.csv", "a") as f:
        first_dict = payload_list[0]
        w = csv.DictWriter(f, first_dict.keys())
        w.writeheader()
        for payload in payload_list:
            w.writerow(payload)


In [11]:
def parse_parameter_map(config_path):
    
    from execs import run_prophet
    
    # Dynamic importing config file from config_path
    path_prefix, filename = split_path(config_path)
    sys.path.insert(0, path_prefix)
    config = importlib.import_module(filename)
    
    # Dynamic loading lambda name
    LAMBDA_NAME = getattr(config.Cross_Validation, "LAMBDA_NAME")

    # Dynamic load parameters 
    PARAMETERS = []
    CV_SETTINGS = []
    for key in dir(config.Hyperparameter):
        if key.isupper():
            PARAMETERS.append(key)
    for key in dir(config.Cross_Validation):
        if key.isupper():
            CV_SETTINGS.append(key)

    # Tune forecast horizon of the chosen model
    create_event(config, PARAMETERS, CV_SETTINGS)

In [2]:
def parse_prophet(file_names):
    
    for file_name in file_names:
        with open("../cloudwatch/log_1549494805.txt", "r") as f:
            line = f.readline()
            while line is not None:


In [ ]:
file_names = os.listdir('../cloudwatch/Prophet/')
parse_prophet(file_names)

In [36]:
parse_parameter_map("/Users/michaelzhang/Downloads/Seneca/config/prophet/config.py")